In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from numpy.random import randint
import math
from torch.nn import functional as F
import numpy as np
import time
import pdb
from torch.autograd import grad
import copy
import shutil

import utils
import models

In [35]:
class opt:
    source_train_path = 'datasets/tomodata/source_train.h5'
    source_test_path = 'datasets/tomodata/source_test.h5'
    target_train_path = 'datasets/tomodata/target_train.h5'
    target_test_path = 'datasets/tomodata/target_test.h5'
    numclass = 4
    fet_size = 128
    h = 400
    nepoch = 50
    nepoch_wdgrl = 50
    cuda = True
    manualSeed = 9182
    batch_size = 32
    tomo_dim = (32,32,32)
    clfr_lr = 0.0001
    critic_iter = 5
    clfr_iter = 5
    gamma = 10
    wd_clf = 1
    save_src_clfr = "best_model/source_clfr.pt"
    save_tar_clfr = "best_model/target_clfr.pt"


In [36]:
if opt.manualSeed is None:
    opt.manualSeed = random.randint(1, 10000)
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)
if opt.cuda:
    torch.cuda.manual_seed_all(opt.manualSeed)

cudnn.benchmark = True

if torch.cuda.is_available() and not opt.cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

Random Seed:  9182


In [37]:
data = utils.DATA_LOADER(opt)
print("# of source training samples: ", data.source_ntrain)
print("# of source test samples: ", data.source_ntest)
print("# of target training samples: ", data.target_ntrain)
print("# of target test samples: ", data.target_ntest)
print("# of class: ", data.numclass)

# of source training samples:  900
# of source test samples:  100
# of target training samples:  900
# of target test samples:  100
# of class:  4


In [38]:
source_clsfr = models.CLFR(opt)
print(source_clsfr)

CLFR(
  (conv1): Sequential(
    (0): Conv3d(1, 8, kernel_size=(5, 5, 5), stride=(1, 1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool3d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv3d(8, 16, kernel_size=(5, 5, 5), stride=(1, 1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv3d(16, 32, kernel_size=(4, 4, 4), stride=(1, 1, 1))
    (1): ReLU(inplace=True)
  )
  (conv4): Sequential(
    (0): Conv3d(32, 64, kernel_size=(4, 4, 4), stride=(1, 1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv5): Sequential(
    (0): Conv3d(64, 128, kernel_size=(2, 2, 2), stride=(1, 1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (hidden): Sequential(
    (0): Linear(in_features=128, out

In [39]:
cls_criterion = nn.CrossEntropyLoss()
input_tomo = torch.FloatTensor(opt.batch_size, opt.tomo_dim[0], opt.tomo_dim[1], opt.tomo_dim[2])
input_label = torch.FloatTensor(opt.batch_size)

In [40]:
if opt.cuda :
    source_clsfr.cuda()
    cls_criterion.cuda()
    input_tomo = input_tomo.cuda()
    input_label = input_label.cuda()

In [41]:
def sample():
    batch_tomo, batch_label = data.source_next_batch(opt.batch_size)
    input_tomo.copy_(batch_tomo)
    input_label.copy_(batch_label)

In [42]:
optim_clfr = optim.Adam(source_clsfr.parameters(), lr=opt.clfr_lr)

In [43]:
def calculate_accuracy(net, tomo, label):
    utils.set_requires_grad(net, requires_grad=False)
    logits = net(tomo.cuda())
    _, predicted = torch.max(logits, 1)
    c = (predicted == label.cuda()).sum()
    c = c.cpu().numpy()
    acc = c / len(label)

    return acc

In [44]:
best_acc = 0

for epoch in range(opt.nepoch):
    for i in range(0, data.source_ntrain, opt.batch_size):

        for p in source_clsfr.parameters():
            p.requires_grad = True

        sample()
        source_clsfr.zero_grad()
        input_tomoV = Variable(input_tomo)
        input_labelV = Variable(input_label.long())

        logits = source_clsfr(input_tomoV)
        loss = cls_criterion(logits, input_labelV)
        loss.backward()
        optim_clfr.step()

    source_clsfr.eval()
    acc = calculate_accuracy(net=source_clsfr, tomo=data.source_test_subtom, label=data.source_test_label)
    print('[%d/%d] clfr_loss: %.4f | test_acc: %.4f'% (epoch, opt.nepoch, loss.item(),acc))

    if acc >= best_acc:
        print('[Source Domain]Validation Acc increased ({:.4f} --> {:.4f}).  Saving model ...'.format(best_acc,acc))
        torch.save(source_clsfr.state_dict(), opt.save_src_clfr)
        best_acc = acc

    source_clsfr.train()

[0/50] clfr_loss: 1.2952 | test_acc: 0.2500
[Source Domain]Validation Acc increased (0.0000 --> 0.2500).  Saving model ...
[1/50] clfr_loss: 1.3515 | test_acc: 0.3300
[Source Domain]Validation Acc increased (0.2500 --> 0.3300).  Saving model ...
[2/50] clfr_loss: 1.1597 | test_acc: 0.3500
[Source Domain]Validation Acc increased (0.3300 --> 0.3500).  Saving model ...
[3/50] clfr_loss: 0.9792 | test_acc: 0.6900
[Source Domain]Validation Acc increased (0.3500 --> 0.6900).  Saving model ...
[4/50] clfr_loss: 0.6006 | test_acc: 0.7500
[Source Domain]Validation Acc increased (0.6900 --> 0.7500).  Saving model ...
[5/50] clfr_loss: 0.3575 | test_acc: 0.8600
[Source Domain]Validation Acc increased (0.7500 --> 0.8600).  Saving model ...
[6/50] clfr_loss: 0.3879 | test_acc: 0.8000
[7/50] clfr_loss: 0.2301 | test_acc: 0.8700
[Source Domain]Validation Acc increased (0.8600 --> 0.8700).  Saving model ...
[8/50] clfr_loss: 0.0988 | test_acc: 0.8900
[Source Domain]Validation Acc increased (0.8700 -->

In [45]:
target_clsfr = models.CLFR(opt)
target_clsfr.load_state_dict(torch.load(opt.save_src_clfr))

<All keys matched successfully>

In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [47]:
def gradient_penalty(critic, h_s, h_t):
    # based on: https://github.com/caogang/wgan-gp/blob/master/gan_cifar10.py#L116
    alpha = torch.rand(h_s.size(0), 1).to(device)
    differences = h_t - h_s
    interpolates = h_s + (alpha * differences)
    interpolates = torch.stack([interpolates, h_s, h_t]).requires_grad_()

    preds = critic(interpolates)
    gradients = grad(preds, interpolates,
                     grad_outputs=torch.ones_like(preds),
                     retain_graph=True, create_graph=True)[0]
    gradient_norm = gradients.norm(2, dim=1)
    gradient_penalty = ((gradient_norm - 1)**2).mean()
    return gradient_penalty

In [48]:
critic = models.CRITIC(opt)
print(critic)

CRITIC(
  (critic_block): Sequential(
    (0): Linear(in_features=128, out_features=50, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)


In [49]:
critic_optim = torch.optim.Adam(critic.parameters(), lr=1e-4)
clf_optim = torch.optim.Adam(target_clsfr.parameters(), lr=opt.clfr_lr)

In [50]:
clf_criterion = nn.CrossEntropyLoss()
source_tomo = torch.FloatTensor(opt.batch_size, opt.tomo_dim[0], opt.tomo_dim[1], opt.tomo_dim[2])
source_label = torch.FloatTensor(opt.batch_size)
target_tomo = torch.FloatTensor(opt.batch_size, opt.tomo_dim[0], opt.tomo_dim[1], opt.tomo_dim[2])

In [51]:
if opt.cuda :
    critic.cuda()
    target_clsfr.cuda()
    clf_criterion.cuda()
    source_tomo = source_tomo.cuda()
    source_label = source_label.cuda()
    target_tomo = target_tomo.cuda()

In [52]:
def both_sample():
    source_batch_tomo, source_batch_label = data.source_next_batch(opt.batch_size)
    source_tomo.copy_(source_batch_tomo)
    source_label.copy_(source_batch_label)
    target_batch_tomo, _ = data.target_next_batch(opt.batch_size)
    target_tomo.copy_(target_batch_tomo)

In [53]:
best_acc = 0

for epoch in range(opt.nepoch_wdgrl):
    for i in range(0, data.source_ntrain, opt.batch_size):
        both_sample()

        #train critic
        utils.set_requires_grad(critic, requires_grad=True)
        utils.set_requires_grad(target_clsfr, requires_grad=False)

        with torch.no_grad():
            h_s = target_clsfr.feature_extractor(source_tomo)
            h_t = target_clsfr.feature_extractor(target_tomo)
        
        for _ in range(opt.critic_iter):
            gp = gradient_penalty(critic, h_s, h_t)
            critic_s = critic(h_s)
            critic_t = critic(h_t)
            wasserstein_distance = critic_s.mean() - critic_t.mean()
            critic_cost = -wasserstein_distance + opt.gamma*gp

            critic_optim.zero_grad()
            critic_cost.backward()
            critic_optim.step()

        #train classifier
        utils.set_requires_grad(critic, requires_grad=False)
        utils.set_requires_grad(target_clsfr, requires_grad=True)

        for _ in range(opt.clfr_iter):
            source_features = target_clsfr.feature_extractor(source_tomo)
            target_features = target_clsfr.feature_extractor(target_tomo)

            source_preds = target_clsfr.classify(source_features)
            clf_loss = clf_criterion(source_preds, source_label.long())

            w_dist = critic(source_features).mean() - critic(target_features).mean()
            loss = clf_loss + opt.wd_clf*w_dist

            clf_optim.zero_grad()
            loss.backward()
            clf_optim.step()

    target_clsfr.eval()

    target_batch_tomo, target_batch_label = data.target_next_batch(100)
    tar_acc = calculate_accuracy(net=target_clsfr, tomo=target_batch_tomo, label=target_batch_label)

    print('[%d/%d] critic_cost: %.4f | w_dist: %.4f | clf_loss: %.4f | tar_acc: %.4f'% (epoch, opt.nepoch_wdgrl, critic_cost.item(), wasserstein_distance.item(), loss.item(), tar_acc))

    if tar_acc >= best_acc:
        print('[Target Domain]Validation Acc increased ({:.4f} --> {:.4f}).  Saving model ...'.format(best_acc,tar_acc))
        torch.save(target_clsfr.state_dict(), opt.save_tar_clfr)
        best_acc = tar_acc

    target_clsfr.train()
        


[0/50] critic_cost: 9.7152 | w_dist: -1.0730 | clf_loss: -1.5050 | tar_acc: 0.9500
[Target Domain]Validation Acc increased (0.0000 --> 0.9500).  Saving model ...
[1/50] critic_cost: 9.9984 | w_dist: -1.4331 | clf_loss: -2.3616 | tar_acc: 0.9500
[Target Domain]Validation Acc increased (0.9500 --> 0.9500).  Saving model ...
[2/50] critic_cost: 9.2756 | w_dist: -0.0461 | clf_loss: -0.0629 | tar_acc: 0.8400
[3/50] critic_cost: 9.0736 | w_dist: 0.1228 | clf_loss: 0.0421 | tar_acc: 0.9600
[Target Domain]Validation Acc increased (0.9500 --> 0.9600).  Saving model ...
[4/50] critic_cost: 8.6616 | w_dist: 0.2328 | clf_loss: 0.0546 | tar_acc: 0.9100
[5/50] critic_cost: 9.9069 | w_dist: -0.9794 | clf_loss: -1.1483 | tar_acc: 0.9400
[6/50] critic_cost: 9.7781 | w_dist: -0.3164 | clf_loss: -0.4664 | tar_acc: 0.8700
[7/50] critic_cost: 10.0536 | w_dist: -0.7349 | clf_loss: -0.8154 | tar_acc: 0.8500
[8/50] critic_cost: 9.0269 | w_dist: 0.4072 | clf_loss: 0.2051 | tar_acc: 0.9100
[9/50] critic_cost: 9